In [1]:
from selenium import webdriver
import selenium
import time

In [2]:
browser = webdriver.Chrome()

In [3]:
browser.get('https://chesstempo.com/chess-players.html')

In [4]:
search_box = browser.find_element_by_css_selector("input#playerName")

In [5]:
search_box.send_keys('Carlsen,Magnus')

In [6]:
search_button = browser.find_element_by_css_selector('#submitbutton')

In [7]:
search_button.click()

In [18]:
#click on first entry 
player = browser.find_element_by_css_selector('#yui-dt0-th-surname .yui-dt-liner')

In [19]:
player.click()